In [1]:
# Google Colab ready notebook
# Paste this into a new Colab cell and run (Runtime -> Run all)
# Installs (only if needed)
!pip install --quiet plotly pandas openpyxl

# Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
from IPython.display import HTML, display

# ---- CONFIG ----
SHEET_ID = "1LmgpJPNVA6bN5_hKIdYC3qoYUdXHr67U"   # from your PDF
PREFERRED_EXPORT_URL = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv"
FALLBACK_EXPORT_URL = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid=0"
OUTPUT_DIR = "/content/olympic_analysis_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Helper to try fetching sheet automatically, otherwise ask for upload
def load_sheet_automatically():
    try:
        print("Trying primary export URL...")
        df = pd.read_csv(PREFERRED_EXPORT_URL)
        print("Loaded using primary URL.")
        return df
    except Exception as e:
        print("Primary failed:", e)
        try:
            print("Trying fallback URL with gid=0...")
            df = pd.read_csv(FALLBACK_EXPORT_URL)
            print("Loaded using fallback URL.")
            return df
        except Exception as e2:
            print("Automatic download failed. Please upload the dataset (Excel/CSV).")
            from google.colab import files
            uploaded = files.upload()
            # Take the first uploaded file
            first_fname = next(iter(uploaded.keys()))
            if first_fname.lower().endswith(".csv"):
                return pd.read_csv(first_fname)
            else:
                return pd.read_excel(first_fname, engine="openpyxl")

# Load data
df_raw = load_sheet_automatically()
print("Raw shape:", df_raw.shape)
display(df_raw.head(3))

# ---- Basic cleaning & standardization ----
df = df_raw.copy()

# Standardize column names: strip and lower replace spaces with _
df.columns = [c.strip().replace(" ", "_").replace("-", "_") for c in df.columns]
# Display columns so you can map if names differ
print("Columns detected:", df.columns.tolist())

# Common column name fixes (adapt if your sheet uses slightly different names)
rename_map = {}
for candidate in ["Edition", "Edition_Year", "Year", "edition"]:
    if candidate in df.columns:
        rename_map[candidate] = "Edition"
for candidate in ["NOC", "Country", "noc"]:
    if candidate in df.columns:
        rename_map[candidate] = "NOC"
for candidate in ["Athlete", "Name", "athlete"]:
    if candidate in df.columns:
        rename_map[candidate] = "Athlete"
for candidate in ["Gender", "Sex", "gender"]:
    if candidate in df.columns:
        rename_map[candidate] = "Gender"
for candidate in ["Sport", "sport"]:
    if candidate in df.columns:
        rename_map[candidate] = "Sport"
for candidate in ["Medal", "medal"]:
    if candidate in df.columns:
        rename_map[candidate] = "Medal"
# apply rename
df = df.rename(columns=rename_map)
print("After rename, columns:", df.columns.tolist())

# Ensure key columns exist; if not, try fuzzy matches
required = ["Edition","NOC","Athlete","Gender","Sport","Medal"]
missing = [r for r in required if r not in df.columns]
if missing:
    print("Warning — these expected columns are missing:", missing)
    print("Try mapping them manually (check detected columns above). Continuing with available columns...")

# Trim strings
for c in df.select_dtypes(include=["object"]).columns:
    df[c] = df[c].astype(str).str.strip()

# Normalize Medal values
if "Medal" in df.columns:
    df['Medal'] = df['Medal'].str.title().replace({"Gold":"Gold","Silver":"Silver","Bronze":"Bronze","GOLD":"Gold","SILVER":"Silver","BRONZE":"Bronze"})
else:
    df['Medal'] = np.nan

# Fix Edition to int where possible
if "Edition" in df.columns:
    df['Edition'] = pd.to_numeric(df['Edition'], errors='coerce').astype('Int64')

# Drop completely empty rows
df = df.dropna(how='all')

# ---- Derived fields ----
# Medal score (Gold=3, Silver=2, Bronze=1)
medal_score_map = {"Gold":3, "Silver":2, "Bronze":1}
df['medal_score'] = df['Medal'].map(medal_score_map).fillna(0).astype(int)

# Athlete aggregated score and medal counts
ath_agg = df.groupby('Athlete', dropna=False).agg(
    total_medals = ('Medal','count'),
    total_score = ('medal_score','sum'),
    golds = ('Medal', lambda x: (x=="Gold").sum()),
    silvers = ('Medal', lambda x: (x=="Silver").sum()),
    bronzes = ('Medal', lambda x: (x=="Bronze").sum())
).reset_index().sort_values('total_medals', ascending=False)

# Create Performance Category (High/Medium/Low) using tertiles of total_score
if ath_agg['total_score'].sum() > 0:
    tertiles = np.quantile(ath_agg['total_score'].replace(0,np.nan).dropna(), [0.33,0.66])
    def perf_cat(score):
        if score >= tertiles[1]:
            return "High"
        elif score >= tertiles[0]:
            return "Medium"
        else:
            return "Low"
    ath_agg['Performance_Category'] = ath_agg['total_score'].apply(lambda s: perf_cat(s) if s>0 else "Low")
else:
    ath_agg['Performance_Category'] = "Low"

# Merge athlete category back to main df (optional)
df = df.merge(ath_agg[['Athlete','Performance_Category']], on='Athlete', how='left')

# ---- Analysis & Visuals ----

# 1) Country-wise Top 10 by medal count
country_counts = df.groupby('NOC', dropna=False).size().reset_index(name='medal_count').sort_values('medal_count', ascending=False)
top10_countries = country_counts.head(10)
print("\nTop 10 countries by medal count:")
display(top10_countries)

fig1 = px.bar(top10_countries, x='NOC', y='medal_count', title="Top 10 NOC by total medals", text='medal_count')
fig1.update_layout(yaxis_title='Medal Count', xaxis_title='NOC')
fig1.write_html(os.path.join(OUTPUT_DIR, "top10_countries.html"))
fig1.show()

# 2) Athlete-level Treemap (top repeat medalists)
top_athletes = ath_agg.head(50)
fig2 = px.treemap(top_athletes, path=['Performance_Category','Athlete'], values='total_medals',
                  color='total_score', title="Top Medal-winning Athletes (treemap)")
fig2.write_html(os.path.join(OUTPUT_DIR, "athlete_treemap.html"))
fig2.show()

# 3) Temporal Gender Analysis (count of unique athletes or medals per Edition split by Gender)
if "Edition" in df.columns and "Gender" in df.columns:
    gender_trend = df.groupby(['Edition','Gender']).size().reset_index(name='count')
    fig3 = px.line(gender_trend, x='Edition', y='count', color='Gender', markers=True, title="Gender trend across Editions (count of medal rows)")
    fig3.update_layout(xaxis=dict(dtick=4))
    fig3.write_html(os.path.join(OUTPUT_DIR, "gender_trend.html"))
    fig3.show()
else:
    print("Skipping gender trend: Edition or Gender column not found.")

# 4) Sport-specific medal trend (stacked by medal type)
if "Sport" in df.columns:
    sport_medal = df.groupby(['Sport','Medal']).size().reset_index(name='count')
    sport_pivot = sport_medal.pivot(index='Sport', columns='Medal', values='count').fillna(0)
    sport_pivot['Total'] = sport_pivot.sum(axis=1)
    sport_pivot = sport_pivot.sort_values('Total', ascending=False).head(20)  # top 20 sports
    sport_pivot = sport_pivot.reset_index()
    fig4 = go.Figure()
    for medal in ["Gold","Silver","Bronze"]:
        if medal in sport_pivot.columns:
            fig4.add_bar(name=medal, x=sport_pivot['Sport'], y=sport_pivot[medal])
    fig4.update_layout(barmode='stack', title="Top 20 Sports by medal counts (stacked by medal type)", xaxis_title="Sport", yaxis_title="Count")
    fig4.write_html(os.path.join(OUTPUT_DIR, "sport_trends.html"))
    fig4.show()
else:
    print("Skipping sport trends: Sport column not found.")

# ---- Key Findings (auto summary) ----
summary_lines = []
# Top NOC
if not country_counts.empty:
    top_noc = country_counts.iloc[0]
    summary_lines.append(f"Top NOC by medals: {top_noc['NOC']} with {top_noc['medal_count']} medals.")
# Rising nations (simple heuristic: check post-1990 counts)
if "Edition" in df.columns and "NOC" in df.columns:
    post90 = df[df['Edition'] >= 1990]
    post_counts = post90.groupby('NOC').size().reset_index(name='post1990_medals').sort_values('post1990_medals', ascending=False).head(5)
    summary_lines.append("Top medal-earning NOCs since 1990 (sample): " + ", ".join([f"{r['NOC']}({r['post1990_medals']})" for _,r in post_counts.iterrows()]))
# Gender representation change (first vs last edition samples)
if "Edition" in df.columns and "Gender" in df.columns:
    editions_sorted = sorted(df['Edition'].dropna().unique())
    if len(editions_sorted) >= 2:
        first, last = editions_sorted[0], editions_sorted[-1]
        first_counts = df[df['Edition']==first].groupby('Gender').size().to_dict()
        last_counts = df[df['Edition']==last].groupby('Gender').size().to_dict()
        summary_lines.append(f"Gender sample: Edition {first} counts {first_counts}; Edition {last} counts {last_counts} (shows growth in female representation).")
# Top sports
if "Sport" in df.columns:
    top_sports = df['Sport'].value_counts().head(6)
    summary_lines.append("Top sports by medal volume: " + ", ".join([f"{s}({c})" for s,c in top_sports.items()]))

summary_text = "\n".join(summary_lines)
print("\nAUTO SUMMARY:")
print(summary_text)

# Save outputs
ath_agg.to_csv(os.path.join(OUTPUT_DIR, "athlete_aggregates.csv"), index=False)
country_counts.to_csv(os.path.join(OUTPUT_DIR, "country_medal_counts.csv"), index=False)
df.to_csv(os.path.join(OUTPUT_DIR, "processed_medals.csv"), index=False)
with open(os.path.join(OUTPUT_DIR, "analysis_summary.txt"), "w", encoding="utf-8") as f:
    f.write(summary_text)

print(f"\nAll outputs saved to {OUTPUT_DIR}. Key interactive charts (html) also saved there.")
# Provide quick file listing
for fname in os.listdir(OUTPUT_DIR):
    print("-", fname)

# Display summary in notebook
display(HTML(f"<h3>Auto summary</h3><pre>{summary_text}</pre>"))


Trying primary export URL...
Loaded using primary URL.
Raw shape: (29216, 10)


,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold
1,Athens,1896,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100m freestyle,M,Silver
2,Athens,1896,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100m freestyle for sailors,M,Bronze


Columns detected: ['City', 'Edition', 'Sport', 'Discipline', 'Athlete', 'NOC', 'Gender', 'Event', 'Event_gender', 'Medal']
After rename, columns: ['City', 'Edition', 'Sport', 'Discipline', 'Athlete', 'NOC', 'Gender', 'Event', 'Event_gender', 'Medal']

Top 10 countries by medal count:


,NOC,medal_count
130,USA,4335
128,URS,2049
43,GBR,1594
41,FRA,1314
62,ITA,1228
46,GER,1211
6,AUS,1075
52,HUN,1053
113,SWE,1021
44,GDR,825



AUTO SUMMARY:
Top NOC by medals: USA with 4335 medals.
Top medal-earning NOCs since 1990 (sample): USA(1311), GER(691), AUS(678), RUS(638), CHN(550)
Gender sample: Edition 1896 counts {'Men': 151}; Edition 2008 counts {'Men': 1110, 'Women': 932} (shows growth in female representation).
Top sports by medal volume: Aquatics(3828), Athletics(3448), Rowing(2523), Gymnastics(2214), Fencing(1547), Football(1387)

All outputs saved to /content/olympic_analysis_output. Key interactive charts (html) also saved there.
- country_medal_counts.csv
- analysis_summary.txt
- top10_countries.html
- athlete_treemap.html
- athlete_aggregates.csv
- sport_trends.html
- processed_medals.csv
- gender_trend.html
